Commençons par charger les données :

In [73]:
import pandas as pd
import numpy as np

data = pd.read_csv('winequality-red.csv', sep=";")

X = data.as_matrix(data.columns[:-1])
y = data.as_matrix([data.columns[-1]])
y = y.flatten()

Séparons nos données en un jeu de test et un jeu d'entraînement :

In [74]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = \
model_selection.train_test_split(X, y,
test_size=0.3 # 30% des données dans le jeu de test
)

Standardisons les données :

In [75]:
from sklearn import preprocessing
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

### 1) Réalisons une régression Knn pour prédire la qualité du vin
#### on utilise SearchGridCV pour rechercher le meilleur hyper-paramètre k du modèle Knn

In [76]:
from sklearn import neighbors

# Créer une régression kNN avec recherche d'hyperparamètre par validation croisée
knn = model_selection.GridSearchCV(neighbors.KNeighborsRegressor(),
    param_grid = {'n_neighbors':[3, 5, 7, 9, 11, 13, 15]}, # hyperparamètres à tester
    cv=5, # nombre de folds de validation croisée
    scoring='neg_mean_squared_error' # choisissons l’erreur quadratique moyenne comme score à optimiser
    )                            


Etant donné que l'algorithme GridSearchCV classe les modèles en maximisant la valeur calculée par "scoring", on est obligé d'utiliser - mean_square_error. En effet un modèle avec une mean_squarred_error élevé est moins bon qu'on modèle avec une mean_squarred_error_ faible.

In [77]:
# Optimiser la régression sur le jeu d'entraînement
knn.fit(X_train_std, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [3, 5, 7, 9, 11, 13, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [78]:
# Afficher le(s) hyperparamètre(s) optimaux
print ("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:",)
print (knn.best_params_)

# Afficher les performances correspondantes
print ("Résultats de la validation croisée :")
for mean, std, params in zip(knn.cv_results_['mean_test_score'], # score moyen
knn.cv_results_['std_test_score'], # écart-type du score
knn.cv_results_['params'] # valeur de l'hyperparamètre
):
    print ("\t%s = %0.3f (+/-%0.03f) for %r" % ('Mean Squarred Error', # critère utilisé
    -mean, # score moyen , on inverse son signe pour afficher mean_squared_error et non pas neg_mean_squared_error
    std * 2, # barre d'erreur
    params # hyperparamètre
    ))
    


Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'n_neighbors': 13}
Résultats de la validation croisée :
	Mean Squarred Error = 0.512 (+/-0.087) for {'n_neighbors': 3}
	Mean Squarred Error = 0.485 (+/-0.073) for {'n_neighbors': 5}
	Mean Squarred Error = 0.458 (+/-0.073) for {'n_neighbors': 7}
	Mean Squarred Error = 0.443 (+/-0.067) for {'n_neighbors': 9}
	Mean Squarred Error = 0.441 (+/-0.061) for {'n_neighbors': 11}
	Mean Squarred Error = 0.438 (+/-0.062) for {'n_neighbors': 13}
	Mean Squarred Error = 0.441 (+/-0.052) for {'n_neighbors': 15}


### Le meilleur hyper-paramètre k pour notre modèle est 13

### 2) Comparons la performance de ce modèle avec celles de deux modèles naîfs

#### une première approche naïve, qui consiste à prédire des valeurs aléatoires

In [79]:
y_pred_random = np.random.randint(np.min(y_train), np.max(y_train), y_test.shape)

#### une deuxième méthode naîve basée sur la valeur moyenne de y

In [80]:
from sklearn import dummy
dum = dummy.DummyRegressor(strategy='mean')

# Entraînement
dum.fit(X_train_std, y_train)

# Prédiction sur le jeu de test
y_pred_dum = dum.predict(X_test_std)


#### Comparons les perfomances des modèles naifs et du modèle knn sur le jeu de test : 

In [81]:
# Afficher les performances des méthodes naives

print ("Résultats de la méthode naîve basée sur une valeur aléatoire :")

print ("Mean Squared Error = ",
metrics.mean_squared_error(y_test, y_pred_random), 
' R2 = ',  
metrics.r2_score(y_test, y_pred_random)                                         
)

print ("Résultats de la méthode naîve basée sur une valeur constante :")

print ("Mean Squared Error = ",
metrics.mean_squared_error(y_test, y_pred_dum), 
' R2 = ',  
metrics.r2_score(y_test, y_pred_dum)                                         
)

y_pred_knn = knn.predict(X_test_std)
print ("Résultats de la méthode knn avec ses meilleurs paramètres :")

print ("Mean Squared Error = ",
metrics.mean_squared_error(y_test, y_pred_knn), 
' R2 = ',  
metrics.r2_score(y_test, y_pred_knn)                                         
)
    

Résultats de la méthode naîve basée sur une valeur aléatoire :
Mean Squared Error =  2.8375  R2 =  -3.3449595916631214
Résultats de la méthode naîve basée sur une valeur constante :
Mean Squared Error =  0.6538205550244736  R2 =  -0.0011714156053188596
Résultats de la méthode knn avec ses meilleurs paramètres :
Mean Squared Error =  0.4469428007889546  R2 =  0.3156128954316306


### 3) Interprétation des valeurs et choix final
le meilleur modèle est celui qui minimise l’erreur quadratique moyenne (MSE). Nous avons donc vérifié que notre modèle knn, même s'il n'est pas excellent, est quand mêm meilleur que les 2 modèles naifs étudiés. Il nous apprend donc quand même quelque chose.

Notre choix porte sur le modèle knn avec k = 13 (remarque : si on relance le notebook, on constate empriquement que la valeur optimale proposée pour k varie entre 11 et 15).